# Databricks Native Excel Reader Demo
- Files: /Volumes/lesteve_sandbox/bronze/sandbox_volume/Sales/2025/
- Sheets: Orders, Targets (title row), ExchangeRates (blank row 2)

In [0]:
base_path = "/Volumes/lesteve_sandbox/bronze/sandbox_volume/Sales/2025/"

##  Discover ALL Sheets in a Workbook

In [0]:
# Discover ALL sheets in a workbook (great for dynamic processing)
sheets_df = spark.read \
  .format("excel") \
  .option("operation", "listSheets") \
  .load(f"{base_path}Sales_2025-01.xlsx")

display(sheets_df)
sheet_names = [row["sheetName"] for row in sheets_df.collect()]
print(f"Sheets found: {sheet_names}")

##  Dynamic Excel workbook and sheet inventory across all Sales_2025 Excel files

In [0]:
from pyspark.sql import functions as F

# List all files in the directory
files_df = spark.read.format("binaryFile").load(base_path)
excel_files = [
    row["path"]
    for row in files_df
        .select("path").distinct()
        .collect()
]

# Collect source_file, sheetIndex, sheetName
all_sheets = []
for file_path in excel_files:
    filename = file_path.split("/")[-1]
    sheets = (
        spark.read
            .format("excel")
            .option("operation", "listSheets")
            .load(file_path)
    )
    for row in sheets.collect():
        all_sheets.append({
            "source_file": filename,
            "sheetIndex": row["sheetIndex"],
            "sheetName": row["sheetName"],
        })

# Display as a simple table
df_all_sheets = spark.createDataFrame(all_sheets)
display(df_all_sheets.orderBy("source_file", "sheetIndex"))


sheetIndex,sheetName,source_file
0,Orders,Sales_2025-01.xlsx
1,Targets,Sales_2025-01.xlsx
2,ExchangeRates,Sales_2025-01.xlsx
0,Orders,Sales_2025-02.xlsx
1,Targets,Sales_2025-02.xlsx
2,ExchangeRates,Sales_2025-02.xlsx
0,Orders,Sales_2025-03.xlsx
1,Notes_ManualAdjustments,Sales_2025-03.xlsx
2,Targets,Sales_2025-03.xlsx
3,ExchangeRates,Sales_2025-03.xlsx


##  Read First Sheet in Specified File by Default

In [0]:
# Reads first sheet (Orders) by default - clean headers at A1
df_default = spark.read \
  .option("headerRows", 1) \
  .option("inferColumnTypes", True) \
  .excel(f"{base_path}Sales_2025-01.xlsx")

display(df_default)
print(f"Rows: {df_default.count()}, Columns: {len(df_default.columns)}")


OrderID,OrderDate,Region,Customer,Product,Quantity,UnitPrice,Currency,TotalAmount
2025-01-0001,2025-01-03T00:00:00,North,Acme Corp,Laptop Pro 15,4,1200,USD,4800
2025-01-0002,2025-01-05T00:00:00,North,Acme Corp,Mouse Wireless,10,25,USD,250
2025-01-0003,2025-01-06T00:00:00,South,Bright Retail,Monitor 27,6,230,USD,1380
2025-01-0004,2025-01-09T00:00:00,EMEA,EuroTech GmbH,Laptop Pro 15,3,1100,EUR,3300
2025-01-0005,2025-01-11T00:00:00,EMEA,EuroTech GmbH,Keyboard Mech,8,70,EUR,560
2025-01-0006,2025-01-15T00:00:00,APAC,Sunrise K.K.,Laptop Air 13,5,950,JPY,4750
2025-01-0007,2025-01-18T00:00:00,APAC,Sunrise K.K.,Mouse Wireless,15,22,JPY,330
2025-01-0008,2025-01-20T00:00:00,South,Bright Retail,Docking Station,3,180,USD,540
2025-01-0009,2025-01-24T00:00:00,North,DataWorks LLC,Monitor 27,4,230,USD,920
2025-01-0010,2025-01-27T00:00:00,EMEA,Nordic Systems,Laptop Air 13,2,1050,EUR,2100


Rows: 10, Columns: 9


##  Read Specific Sheet in a File

In [0]:
# Read SPECIFIC sheet (Orders) - clean headers at A1
df_default = spark.read \
  .option("headerRows", 1) \
  .option("inferColumnTypes", True) \
  .option("dataAddress", "Orders") \
  .excel(f"{base_path}Sales_2025-01.xlsx")

display(df_default)
print(f"Rows: {df_default.count()}, Columns: {len(df_default.columns)}")


OrderID,OrderDate,Region,Customer,Product,Quantity,UnitPrice,Currency,TotalAmount
2025-01-0001,2025-01-03T00:00:00,North,Acme Corp,Laptop Pro 15,4,1200,USD,4800
2025-01-0002,2025-01-05T00:00:00,North,Acme Corp,Mouse Wireless,10,25,USD,250
2025-01-0003,2025-01-06T00:00:00,South,Bright Retail,Monitor 27,6,230,USD,1380
2025-01-0004,2025-01-09T00:00:00,EMEA,EuroTech GmbH,Laptop Pro 15,3,1100,EUR,3300
2025-01-0005,2025-01-11T00:00:00,EMEA,EuroTech GmbH,Keyboard Mech,8,70,EUR,560
2025-01-0006,2025-01-15T00:00:00,APAC,Sunrise K.K.,Laptop Air 13,5,950,JPY,4750
2025-01-0007,2025-01-18T00:00:00,APAC,Sunrise K.K.,Mouse Wireless,15,22,JPY,330
2025-01-0008,2025-01-20T00:00:00,South,Bright Retail,Docking Station,3,180,USD,540
2025-01-0009,2025-01-24T00:00:00,North,DataWorks LLC,Monitor 27,4,230,USD,920
2025-01-0010,2025-01-27T00:00:00,EMEA,Nordic Systems,Laptop Air 13,2,1050,EUR,2100


Rows: 10, Columns: 9


##  Read specific Sheet and Data Range in a File

In [0]:
# Targets has title "Global Sales Targets" in row 1, headers in row 2
df_targets = spark.read \
  .option("headerRows", 1) \
  .option("dataAddress", "'Targets'!A2:D10") \
  .excel(f"{base_path}Sales_2025-01.xlsx")

display(df_targets)


Month,Region,Product,TargetRevenue
2025-01-01T00:00:00,North,Laptop Pro 15,50000
2025-01-01T00:00:00,North,Monitor 27,30000
2025-01-01T00:00:00,South,Laptop Pro 15,25000
2025-01-01T00:00:00,South,Monitor 27,20000
2025-01-01T00:00:00,EMEA,Laptop Air 13,40000
2025-01-01T00:00:00,EMEA,Keyboard Mech,15000
2025-01-01T00:00:00,APAC,Laptop Air 13,35000
2025-01-01T00:00:00,APAC,Mouse Wireless,10000


In [0]:
# Read SPECIFIC sheet (ExchangeRates) - exclude null rows
from pyspark.sql import functions as F
df_default = spark.read \
  .option("headerRows", 1) \
  .option("inferColumnTypes", True) \
  .option("dataAddress", "ExchangeRates") \
  .excel(f"{base_path}Sales_2025-01.xlsx") \
  .dropna(how="all")

display(df_default)
print(f"Rows: {df_default.count()}, Columns: {len(df_default.columns)}")


RateDate,Currency,FxToUSD
2025-01-01T00:00:00,USD,1.0
2025-01-01T00:00:00,EUR,1.1
2025-01-01T00:00:00,JPY,0.009
2025-01-15T00:00:00,EUR,1.08
2025-01-15T00:00:00,JPY,0.0085


Rows: 5, Columns: 3


In [0]:
# Read SPECIFIC sheet (Orders) and range containing Product & Quantity E1:F10 - clean headers at A1
df_default = spark.read \
  .option("headerRows", 1) \
  .option("inferColumnTypes", True) \
  .option("dataAddress", "'Orders'!E1:F10") \
  .excel(f"{base_path}Sales_2025-01.xlsx")

display(df_default)
print(f"Rows: {df_default.count()}, Columns: {len(df_default.columns)}")


Product,Quantity
Laptop Pro 15,4
Mouse Wireless,10
Monitor 27,6
Laptop Pro 15,3
Keyboard Mech,8
Laptop Air 13,5
Mouse Wireless,15
Docking Station,3
Monitor 27,4


Rows: 9, Columns: 2


## Dynamic ingestion and cleansing of all Orders sheets from every Excel file in the Sales_2025 folder

In [0]:
# Dynamic "Orders" sheet processing - auto-detects range, skips null rows/cols
from pyspark.sql import functions as F
import functools

# List all Excel files in the directory
files_df = spark.read.format("binaryFile").load(base_path)
excel_files = [
    row["path"]
    for row in files_df
        .filter(F.col("path").endswith(".xlsx"))
        .select("path").distinct()
        .collect()
]

all_orders_dfs = []
for file_path in excel_files:
    month_file = file_path.split("/")[-1]
    # List sheets first
    sheets_df = spark.read.format("excel").option("operation", "listSheets").load(file_path)
    orders_sheet = [s["sheetName"] for s in sheets_df.collect() if s["sheetName"] == "Orders"]
    
    if orders_sheet:
        sheet_name = orders_sheet[0]
        
        # Read WHOLE Orders sheet, clean nulls
        df_month = spark.read \
          .option("headerRows", 1) \
          .option("inferColumnTypes", True) \
          .option("dataAddress", sheet_name) \
          .excel(file_path) \
          .withColumn("source_month", F.lit(month_file))
        
        # Remove fully null rows
        df_month = df_month.dropna(how="all")
        
        # Remove fully null columns
        non_null_cols = [c for c in df_month.columns if df_month.filter(F.col(c).isNotNull()).count() > 0]
        if non_null_cols:
            df_month = df_month.select(non_null_cols)
        
        all_orders_dfs.append(df_month)

# Union all Orders sheets
if all_orders_dfs:
    df_unified_orders = functools.reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), all_orders_dfs)
    # Remove fully null rows and columns from the final df
    df_unified_orders = df_unified_orders.dropna(how="all")
    non_null_cols_final = [c for c in df_unified_orders.columns if df_unified_orders.filter(F.col(c).isNotNull()).count() > 0]
    df_unified_orders = df_unified_orders.select(non_null_cols_final)
    display(df_unified_orders)

OrderID,OrderDate,Region,Customer,Product,Quantity,UnitPrice,Currency,TotalAmount,source_month
2025-03-0001,2025-03-01T00:00:00,North,Acme Corp,Laptop Pro 15,6,1175,USD,7050,Sales_2025-03.xlsx
2025-03-0002,2025-03-03T00:00:00,North,DataWorks LLC,Mouse Wireless,14,24,USD,336,Sales_2025-03.xlsx
2025-03-0003,2025-03-06T00:00:00,South,Bright Retail,Monitor 27,8,238,USD,1904,Sales_2025-03.xlsx
2025-03-0004,2025-03-09T00:00:00,EMEA,EuroTech GmbH,Laptop Pro 15,5,1130,EUR,5650,Sales_2025-03.xlsx
2025-03-0005,2025-03-12T00:00:00,EMEA,Nordic Systems,Keyboard Mech,9,73,EUR,657,Sales_2025-03.xlsx
2025-03-0006,2025-03-15T00:00:00,APAC,Sunrise K.K.,Laptop Air 13,7,965,JPY,6755,Sales_2025-03.xlsx
2025-03-0007,2025-03-18T00:00:00,APAC,Pacific Retail,Mouse Wireless,20,23,JPY,460,Sales_2025-03.xlsx
2025-03-0008,2025-03-20T00:00:00,South,Bright Retail,Docking Station,5,188,USD,940,Sales_2025-03.xlsx
2025-03-0009,2025-03-23T00:00:00,North,DataWorks LLC,Monitor 27,6,238,USD,1428,Sales_2025-03.xlsx
2025-03-0010,2025-03-26T00:00:00,EMEA,Nordic Systems,Laptop Air 13,4,1070,EUR,4280,Sales_2025-03.xlsx


## Read Excel Sheet and Data Range from SharePoint connection

In [0]:
from pyspark.sql import functions as F

df = (spark.read
        .format("excel")
        .option("databricks.connection", "sharepoint_oauth")
        .option("headerRows", 1)
        .option("dataAddress", "'Orders'!A1:I10")
        .load("https://myorg.sharepoint.com/sites/SharedServices/Shared%20Documents/Sample%20Data/Sales_2025/Sales_2025-01.xlsx"))
display(df)

OrderID,OrderDate,Region,Customer,Product,Quantity,UnitPrice,Currency,TotalAmount
2025-01-0001,2025-01-03T00:00:00,North,Acme Corp,Laptop Pro 15,4,1200,USD,4800
2025-01-0002,2025-01-05T00:00:00,North,Acme Corp,Mouse Wireless,10,25,USD,250
2025-01-0003,2025-01-06T00:00:00,South,Bright Retail,Monitor 27,6,230,USD,1380
2025-01-0004,2025-01-09T00:00:00,EMEA,EuroTech GmbH,Laptop Pro 15,3,1100,EUR,3300
2025-01-0005,2025-01-11T00:00:00,EMEA,EuroTech GmbH,Keyboard Mech,8,70,EUR,560
2025-01-0006,2025-01-15T00:00:00,APAC,Sunrise K.K.,Laptop Air 13,5,950,JPY,4750
2025-01-0007,2025-01-18T00:00:00,APAC,Sunrise K.K.,Mouse Wireless,15,22,JPY,330
2025-01-0008,2025-01-20T00:00:00,South,Bright Retail,Docking Station,3,180,USD,540
2025-01-0009,2025-01-24T00:00:00,North,DataWorks LLC,Monitor 27,4,230,USD,920


## Dynamically Read all SharePoint Excel Files containing data in the 'Order' tabs in the specified folder.

In [0]:
from pyspark.sql import functions as F
import functools

# SharePoint connection
sharepoint_base = "https://myorg.sharepoint.com/sites/SharedServices/Shared%20Documents/Sample%20Data/Sales_2025/"

# Explicitly list the Excel files to process (SharePoint HTTPS doesn't support directory listing)
# You can retrieve this list from SharePoint API or maintain it manually
excel_filenames = [
    "Sales_2025-01.xlsx",
    "Sales_2025-02.xlsx",
    "Sales_2025-03.xlsx"
]

sp_excel_files = [sharepoint_base + filename for filename in excel_filenames]

all_sp_orders_dfs = []
for sp_file in sp_excel_files:
    month_file = sp_file.split("/")[-1]
    
    try:
        # List sheets in the file
        sheets_df = spark.read.format("excel") \
            .option("databricks.connection", "sharepoint_oauth") \
            .option("operation", "listSheets") \
            .load(sp_file)
        orders_sheet = [s["sheetName"] for s in sheets_df.collect() if s["sheetName"] == "Orders"]
        
        if orders_sheet:
            sheet_name = orders_sheet[0]
            # Read WHOLE Orders sheet, clean nulls
            df_sp = spark.read \
                .format("excel") \
                .option("databricks.connection", "sharepoint_oauth") \
                .option("headerRows", 1) \
                .option("inferColumnTypes", True) \
                .option("dataAddress", sheet_name) \
                .load(sp_file) \
                .withColumn("source_file", F.lit(month_file))
            
            # Remove fully null rows
            df_sp = df_sp.dropna(how="all")
            # Remove fully null columns
            non_null_cols = [c for c in df_sp.columns if df_sp.filter(F.col(c).isNotNull()).count() > 0]
            if non_null_cols:
                df_sp = df_sp.select(non_null_cols)
            all_sp_orders_dfs.append(df_sp)
            print(f"✓ Processed {month_file}")
    except Exception as e:
        print(f"✗ Skipped {month_file}: {str(e)}")

# Union all SharePoint Orders sheets → DataFrame only
if all_sp_orders_dfs:
    df_sp_unified = functools.reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), all_sp_orders_dfs)
    df_sp_unified = df_sp_unified.dropna(how="all")
    non_null_cols_final = [c for c in df_sp_unified.columns if df_sp_unified.filter(F.col(c).isNotNull()).count() > 0]
    df_sp_unified = df_sp_unified.select(non_null_cols_final)
    print(f"\n✓ Combined {len(all_sp_orders_dfs)} files with {df_sp_unified.count()} total rows")
    display(df_sp_unified)
else:
    print("No Orders sheets found in any files")

✓ Processed Sales_2025-01.xlsx
✓ Processed Sales_2025-02.xlsx
✓ Processed Sales_2025-03.xlsx

✓ Combined 3 files with 30 total rows


OrderID,OrderDate,Region,Customer,Product,Quantity,UnitPrice,Currency,TotalAmount,source_file
2025-01-0001,2025-01-03T00:00:00,North,Acme Corp,Laptop Pro 15,4,1200,USD,4800,Sales_2025-01.xlsx
2025-01-0002,2025-01-05T00:00:00,North,Acme Corp,Mouse Wireless,10,25,USD,250,Sales_2025-01.xlsx
2025-01-0003,2025-01-06T00:00:00,South,Bright Retail,Monitor 27,6,230,USD,1380,Sales_2025-01.xlsx
2025-01-0004,2025-01-09T00:00:00,EMEA,EuroTech GmbH,Laptop Pro 15,3,1100,EUR,3300,Sales_2025-01.xlsx
2025-01-0005,2025-01-11T00:00:00,EMEA,EuroTech GmbH,Keyboard Mech,8,70,EUR,560,Sales_2025-01.xlsx
2025-01-0006,2025-01-15T00:00:00,APAC,Sunrise K.K.,Laptop Air 13,5,950,JPY,4750,Sales_2025-01.xlsx
2025-01-0007,2025-01-18T00:00:00,APAC,Sunrise K.K.,Mouse Wireless,15,22,JPY,330,Sales_2025-01.xlsx
2025-01-0008,2025-01-20T00:00:00,South,Bright Retail,Docking Station,3,180,USD,540,Sales_2025-01.xlsx
2025-01-0009,2025-01-24T00:00:00,North,DataWorks LLC,Monitor 27,4,230,USD,920,Sales_2025-01.xlsx
2025-01-0010,2025-01-27T00:00:00,EMEA,Nordic Systems,Laptop Air 13,2,1050,EUR,2100,Sales_2025-01.xlsx
